### Extracción de características a nivel de clip (UCF-Crime)

Este notebook utiliza como entrada el archivo `processed/index_clips.csv`

El archivo `index_clips.csv` contiene, para cada clip temporal:
- La ruta al video original.
- El rango temporal del clip (`start_frame`, `end_frame`).
- La partición correspondiente (`train`, `val`, `test`).
- La etiqueta binaria (normal vs anómalo) y la categoría asociada.
- Los parámetros de segmentación utilizados (longitud del clip y solapamiento).

A partir de este índice, se cargan los frames correspondientes a cada clip y se transforman en la
representación requerida por los modelos evaluados, sin redefinir ni modificar la composición del
conjunto experimental.


In [1]:
import pandas as pd
from pathlib import Path
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Ruta al índice de clips
INDEX_CLIPS_PATH = Path("processed/index_clips.csv")

# Verificar existencia
assert INDEX_CLIPS_PATH.exists(), f"No se encuentra el archivo: {INDEX_CLIPS_PATH}"

# Cargar CSV
df_clips = pd.read_csv(INDEX_CLIPS_PATH)

print("Archivo cargado correctamente")
print("Número total de clips:", len(df_clips))

display(df_clips.head())


Archivo cargado correctamente
Número total de clips: 145356


,split,y,category,path,clip_idx,start_frame,end_frame,clip_len,stride,fps,n_frames
0,train,0,Normal,/home/DIINF/dvaldes/tesis/UCF_Crime/Training-N...,0,0,32,32,16,30.0,2016
1,train,0,Normal,/home/DIINF/dvaldes/tesis/UCF_Crime/Training-N...,1,16,48,32,16,30.0,2016
2,train,0,Normal,/home/DIINF/dvaldes/tesis/UCF_Crime/Training-N...,2,32,64,32,16,30.0,2016
3,train,0,Normal,/home/DIINF/dvaldes/tesis/UCF_Crime/Training-N...,3,48,80,32,16,30.0,2016
4,train,0,Normal,/home/DIINF/dvaldes/tesis/UCF_Crime/Training-N...,4,64,96,32,16,30.0,2016


In [2]:
# Distribución por split
print("Clips por split:")
print(df_clips["split"].value_counts())

# Distribución por clase
print("\nClips por clase (y):")
print(df_clips["y"].value_counts())

# Verificar rangos temporales válidos
invalid_ranges = df_clips[df_clips["end_frame"] <= df_clips["start_frame"]]
print("\nClips con rangos inválidos:", len(invalid_ranges))

# Verificar paths únicos y existencia
missing_paths = df_clips[~df_clips["path"].apply(lambda p: Path(p).exists())]
print("Clips con path inexistente:", len(missing_paths))


Clips por split:
split
train    106527
val       19793
test      19036
Name: count, dtype: int64

Clips por clase (y):
y
0    73870
1    71486
Name: count, dtype: int64

Clips con rangos inválidos: 0
Clips con path inexistente: 0


In [3]:
# Verificar que un mismo video no aparezca en más de un split
video_split_counts = df_clips.groupby("path")["split"].nunique()
n_leak = int((video_split_counts > 1).sum())

print("Videos con clips en más de un split:", n_leak)


Videos con clips en más de un split: 0


# Carga y congelamiento del encoder VideoCLIP (X-CLIP)

Objetivo:
Cargar un modelo preentrenado tipo VideoCLIP (X-CLIP), moverlo a GPU/CPU según disponibilidad y congelar sus parámetros para utilizarlo como feature extractor.

Entradas:

VIDEOCLIP_CKPT: string con el identificador del checkpoint (p. ej., "microsoft/xclip-base-patch32").

DEVICE: dispositivo de cómputo seleccionado automáticamente ("cuda" o "cpu").

Salidas:

processor: XCLIPProcessor con configuración de preprocesamiento (incluye image_mean y image_std).

encoder: XCLIPModel preentrenado en modo evaluación y con parámetros congelados (requires_grad=False).

Impresiones en consola: dispositivo seleccionado, checkpoint cargado y valores mean/std para normalización.

Descripción técnica:
Se inicializa el procesador asociado al checkpoint y se carga el modelo X-CLIP. Luego se activa modo eval() para deshabilitar capas dependientes de entrenamiento (p. ej., dropout) y se congelan los parámetros para evitar actualización durante el entrenamiento del clasificador posterior (MLP).


In [4]:
import torch
from transformers import XCLIPModel, XCLIPProcessor

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

VIDEOCLIP_CKPT = "microsoft/xclip-base-patch32"

processor = XCLIPProcessor.from_pretrained(VIDEOCLIP_CKPT)

encoder = XCLIPModel.from_pretrained(VIDEOCLIP_CKPT).to(DEVICE)
encoder.eval()

for p in encoder.parameters():
    p.requires_grad = False

print("OK loaded:", VIDEOCLIP_CKPT)
print(
    "processor mean/std:",
    processor.image_processor.image_mean,
    processor.image_processor.image_std
)



DEVICE: cuda


/home/DIINF/dvaldes/venvs/tesis/lib/python3.10/site-packages/torch/cuda/__init__.py:827: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


OK loaded: microsoft/xclip-base-patch32
processor mean/std: [0.485, 0.456, 0.406] [0.229, 0.224, 0.225]


Definición del input y Dataset de clips (muestreo temporal + preprocesamiento)

Objetivo:
Construir un Dataset que, a partir de un índice de clips (rango de frames por video), cargue T frames uniformemente muestreados, los preprocese (RGB, resize, normalización) y devuelva un tensor listo para el encoder.

Entradas:

T: número de frames por clip (p. ej., 8).

IMG_SIZE: tamaño espacial objetivo (p. ej., 224×224).

df: DataFrame con al menos: path, start_frame, end_frame, y.

processor.image_processor.image_mean/std: estadísticas de normalización del checkpoint X-CLIP.

Salidas:

clip: tensor torch.float32 con forma (C, T, H, W) = (3, T, 224, 224).

y: etiqueta torch.long (0/1).

Descripción técnica:
Para cada fila del índice, se abre el video con OpenCV y se seleccionan T frames distribuidos uniformemente entre start_frame y end_frame. Cada frame se convierte a RGB, se redimensiona a IMG_SIZE y se normaliza usando mean/std del modelo (CLIP-style). El clip resultante se organiza en el formato (C, T, H, W), que luego se reordena a (B, T, C, H, W) antes de ingresar al encoder.

Nota de implementación (reproducibilidad):
Este bloque asume que la variable global processor ya fue creada previamente. Para mayor robustez, se recomienda pasar mean/std como parámetros al Dataset y evitar dependencias globales.

In [17]:

# Parámetros del input
T = 8                
IMG_SIZE = 224       
BATCH_SIZE = 16
NUM_WORKERS = 8

def uniform_sample_indices(start_f: int, end_f: int, T: int):
    n = max(1, end_f - start_f)
    idx = np.linspace(0, n - 1, T).round().astype(int)
    return (start_f + idx).astype(int)

class ClipDataset(Dataset):
    def __init__(self, df, T=8, img_size=224):
        self.df = df.reset_index(drop=True)
        self.T = T
        self.img_size = img_size
        self.mean = np.array(processor.image_processor.image_mean, dtype=np.float32)
        self.std  = np.array(processor.image_processor.image_std, dtype=np.float32)


    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        path = row["path"]
        start_f = int(row["start_frame"])
        end_f   = int(row["end_frame"])
        y = int(row["y"])

        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            raise RuntimeError(f"No pude abrir video: {path}")

        frame_ids = uniform_sample_indices(start_f, end_f, self.T)

        frames = []
        last_good = None
        for fid in frame_ids:
            cap.set(cv2.CAP_PROP_POS_FRAMES, int(fid))
            ok, frame = cap.read()

            if not ok:
                if last_good is None:
                    frame = np.zeros((self.img_size, self.img_size, 3), dtype=np.uint8)
                else:
                    frame = last_good
            else:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (self.img_size, self.img_size), interpolation=cv2.INTER_LINEAR)
                last_good = frame

            frames.append(frame)

        cap.release()

        # (T,H,W,C) -> float [0,1]
        arr = np.stack(frames).astype(np.float32) / 255.0
        arr = (arr - self.mean) / self.std

        # -> (C,T,H,W)
        arr = np.transpose(arr, (3, 0, 1, 2))
        clip = torch.from_numpy(arr)  # float32

        return clip, torch.tensor(y, dtype=torch.long)


In [18]:
df_train = df_clips[df_clips["split"]=="train"].copy()
df_val   = df_clips[df_clips["split"]=="val"].copy()
df_test  = df_clips[df_clips["split"]=="test"].copy()

train_ds = ClipDataset(df_train, T=T, img_size=IMG_SIZE)
val_ds   = ClipDataset(df_val,   T=T, img_size=IMG_SIZE)
test_ds  = ClipDataset(df_test,  T=T, img_size=IMG_SIZE)

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

test_loader = DataLoader(
    test_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

xb, yb = next(iter(train_loader))
print("xb shape:", xb.shape)  # (B, 3, T, 224, 224)
print("yb shape:", yb.shape)  # (B,)


xb shape: torch.Size([16, 3, 8, 224, 224])
yb shape: torch.Size([16])


Extracción de embeddings de video (modo unimodal con X-CLIP)

Objetivo:
Obtener un embedding vectorial fijo por clip de video utilizando únicamente la rama visual del modelo X-CLIP.

Entradas:

encoder: modelo XCLIPModel preentrenado y congelado.

pixel_values: tensor con forma (B, T, C, H, W), donde:

B = batch size

T = número de frames

C = canales (3)

H, W = dimensiones espaciales (224×224)

Salidas:

video_embeds: tensor con forma (B, D), donde D = 512 (dimensión del embedding del modelo base).

Descripción técnica:
Cada frame del clip se procesa individualmente mediante el encoder visual (vision_model).
Las representaciones resultantes se proyectan al espacio CLIP (visual_projection) para obtener embeddings por frame.
Posteriormente, estos embeddings se reorganizan en secuencia temporal (B, T, D) y se integran mediante el módulo temporal MIT del modelo, produciendo un embedding global por clip.

El bloque final de uso:

Reordena el tensor de entrada de (B, C, T, H, W) a (B, T, C, H, W).

Ejecuta la extracción bajo torch.no_grad() para evitar cálculo de gradientes.

Verifica dimensionalidades esperadas.

Este embedding es el que posteriormente alimenta el clasificador MLP para la tarea de detección binaria (normal/anómalo).

In [19]:
import torch

def xclip_video_embeds_unimodal(encoder, pixel_values: torch.Tensor):
    """
    pixel_values: (B, T, C, H, W)
    return: (B, D) video embedding
    """
    B, T, C, H, W = pixel_values.shape

    # 1) Aplanar frames: (B*T, C, H, W)
    flat = pixel_values.reshape(B * T, C, H, W).contiguous()

    # 2) Vision encoder por frame
    vision_out = encoder.vision_model(pixel_values=flat)

    if isinstance(vision_out, (tuple, list)):
        frame_pooled = vision_out[1]
    else:
        frame_pooled = vision_out.pooler_output

    # 3) Proyección (emb por frame)
    frame_embeds = encoder.visual_projection(frame_pooled)  # (B*T, D)

    # 4) Secuencia temporal: (B, T, D)
    cls_features = frame_embeds.view(B, T, -1)

    # 5) Integración temporal (MIT)
    mit_out = encoder.mit(cls_features)

    if isinstance(mit_out, (tuple, list)):
        video_embeds = mit_out[1] if len(mit_out) > 1 else mit_out[0]
    else:
        video_embeds = mit_out.pooler_output

    return video_embeds


# ====== USO ======
xb = xb.to(DEVICE)
pixel_values = xb.permute(0, 2, 1, 3, 4).contiguous()  # (B, T, C, H, W)

with torch.no_grad():
    video_embeds = xclip_video_embeds_unimodal(encoder, pixel_values)

print("pixel_values shape:", pixel_values.shape)
print("video_embeds shape:", video_embeds.shape)  # (B, 512)


pixel_values shape: torch.Size([16, 8, 3, 224, 224])
video_embeds shape: torch.Size([16, 512])


In [20]:
with torch.no_grad():
    video_embeds = xclip_video_embeds_unimodal(encoder, pixel_values)

print("video_embeds shape:", video_embeds.shape)  # (B, 512)


video_embeds shape: torch.Size([16, 512])


# 3 Extracción de emebdings


In [ ]:
import numpy as np
from pathlib import Path

def create_memmap(path, shape, dtype="float16"):
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    return np.memmap(path, mode="w+", dtype=dtype, shape=shape)


In [ ]:
# ejemplo con un batch dummy ya calculado
D = int(video_embeds.shape[1])   # debería ser 512
print("Embedding dim:", D)



In [ ]:
df_train = df_clips[df_clips["split"]=="train"].copy()
df_val   = df_clips[df_clips["split"]=="val"].copy()
df_test  = df_clips[df_clips["split"]=="test"].copy()

train_ds = ClipDataset(df_train, T=T, img_size=IMG_SIZE)
val_ds   = ClipDataset(df_val,   T=T, img_size=IMG_SIZE)
test_ds  = ClipDataset(df_test,  T=T, img_size=IMG_SIZE)

print(len(train_ds), len(val_ds), len(test_ds))


In [ ]:

# Train
X_train = create_memmap("processed/emb_xclip_video_train.mmap", (len(train_ds), D), dtype="float16")
y_train = create_memmap("processed/y_train.mmap", (len(train_ds),), dtype="int8")

# Val
X_val = create_memmap("processed/emb_xclip_video_val.mmap", (len(val_ds), D), dtype="float16")
y_val = create_memmap("processed/y_val.mmap", (len(val_ds),), dtype="int8")

# Test
X_test = create_memmap("processed/emb_xclip_video_test.mmap", (len(test_ds), D), dtype="float16")
y_test = create_memmap("processed/y_test.mmap", (len(test_ds),), dtype="int8")



In [ ]:
import numpy as np
from pathlib import Path
from tqdm import tqdm
import torch

def create_memmap(path, shape, dtype="float16"):
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    return np.memmap(path, mode="w+", dtype=dtype, shape=shape)

def extract_embeddings_xclip(loader, encoder, X_mm, y_mm, split_name="train"):
    encoder.eval()

    total_batches = len(loader)
    print(f"\nExtracting {split_name.upper()} embeddings (X-CLIP)...")
    print(f"Total batches: {total_batches}")

    with torch.no_grad():
        offset = 0

        for xb, yb in tqdm(loader, desc=f"{split_name}", leave=True):
            xb = xb.to(DEVICE)  # (B, C, T, H, W)
            pixel_values = xb.permute(0, 2, 1, 3, 4).contiguous()  # (B, T, C, H, W)

            # ✅ embedding (B, 512)
            video_embeds = xclip_video_embeds_unimodal(encoder, pixel_values)

            emb_np = video_embeds.detach().cpu().numpy().astype(X_mm.dtype, copy=False)
            y_np = yb.numpy().astype(y_mm.dtype, copy=False)

            bs = emb_np.shape[0]
            X_mm[offset:offset+bs] = emb_np
            y_mm[offset:offset+bs] = y_np
            offset += bs

    X_mm.flush()
    y_mm.flush()
    print(f"{split_name.upper()} extraction done.")



In [ ]:
extract_embeddings_xclip(train_loader, encoder, X_train, y_train, split_name="train")
extract_embeddings_xclip(val_loader,   encoder, X_val,   y_val,   split_name="val")
extract_embeddings_xclip(test_loader,  encoder, X_test,  y_test,  split_name="test")

print("\nAll embeddings extracted successfully.")


In [ ]:
import numpy as np

print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)


In [ ]:
import numpy as np

def sanity_mm_fp32(X_mm, y_mm, name="split", n=5000):
    n = min(n, len(y_mm))
    X = np.array(X_mm[:n], dtype=np.float32)   # <- clave
    y = np.array(y_mm[:n], dtype=np.int64)

    print(f"\n[{name}]")
    print("  finite:", np.isfinite(X).all())
    print("  mean:", float(X.mean()))
    print("  std:",  float(X.std()))
    print("  min/max:", float(X.min()), float(X.max()))
    print("  y counts:", {int(v): int((y==v).sum()) for v in np.unique(y)})

sanity_mm_fp32(X_train, y_train, "train")
sanity_mm_fp32(X_val,   y_val,   "val")
sanity_mm_fp32(X_test,  y_test,  "test")



In [ ]:
X = np.array(X_train[:5000], dtype=np.float32)
print("Any inf:", np.isinf(X).any())
print("Any nan:", np.isnan(X).any())


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# usa una muestra pequeña para que sea rápido
ntr = min(20000, len(y_train))
nva = min(8000,  len(y_val))

Xtr = np.array(X_train[:ntr], dtype=np.float32)
ytr = np.array(y_train[:ntr], dtype=np.int64)

Xva = np.array(X_val[:nva], dtype=np.float32)
yva = np.array(y_val[:nva], dtype=np.int64)

clf = LogisticRegression(max_iter=200, n_jobs=-1)
clf.fit(Xtr, ytr)

pva = clf.predict_proba(Xva)[:, 1]
auc = roc_auc_score(yva, pva)
print("Sanity AUC (LogReg):", auc)


In [ ]:
import json
from datetime import datetime
from pathlib import Path

manifest = {
    "created_at": datetime.now().isoformat(),
    "model": VIDEOCLIP_CKPT,  # <-- cambia esto
    "T": T,
    "img_size": IMG_SIZE,
    "embedding_dim": int(X_train.shape[1]),  # debería ser 512
    "files": {
        "X_train": "processed/emb_xclip_video_train.mmap",
        "y_train": "processed/y_train.mmap",
        "X_val":   "processed/emb_xclip_video_val.mmap",
        "y_val":   "processed/y_val.mmap",
        "X_test":  "processed/emb_xclip_video_test.mmap",
        "y_test":  "processed/y_test.mmap",
    }
}

Path("processed").mkdir(exist_ok=True)

with open("processed/manifest_xclip_video.json", "w") as f:
    json.dump(manifest, f, indent=2)

print("Saved:", "processed/manifest_xclip_video.json")
